In [1]:
import findspark
import sys
findspark.init('/opt/spark')
print("Python version:", sys.version)

Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
    .enableHiveSupport()\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark.sql("DROP TABLE IF EXISTS posts")

DataFrame[]

In [5]:
flickrData=spark.read.json("hdfs://master/user/custom/allFlickrRome.json")

In [6]:
# Create Hive Internal table
flickrData.write.mode("overwrite").saveAsTable("posts")

In [7]:
spark.sql("DROP FUNCTION IF EXISTS GeoData")
spark.sql("DROP FUNCTION IF EXISTS GeoDbscan")
spark.sql("ADD JAR hdfs://master/user/custom/HiveUDF-1.0-SNAPSHOT-jar-with-dependencies.jar")
spark.sql("CREATE FUNCTION GeoData AS 'it.unical.dimes.scalab.hive.udf.GeoDataUDF'");
spark.sql("CREATE FUNCTION GeoDbscan AS 'it.unical.dimes.scalab.hive.udf.DbScanUDAF'");

In [8]:
keywords = "colosseo,colis,collis,collos,Amphiteatrum Flavium,Amphitheatrum Flavium,An Colasaem,Coliseo,Coliseo,Coliseo de Roma,Coliseo de Roma,Coliseu de Roma,Coliseum,Coliseum,Coliseum,Coliseus,Colloseum,Coloseu,Colosseo,Colosseo,Colosseo,Colosseu,Colosseum,Colosseum,Colosseum,Colosseum,Colosseum,Colosseum,Colosseum,Colosseum,Colosseum,Colosseum,Culusseu,Kolezyum,Koliseoa,Kolize,Kolizejs,Kolizey,Kolizey,Koliziejus,Kolosej,Kolosej,Koloseo,Koloseo,Koloseum,Koloseum,Koloseum,Koloseum,Koloseum,Koloseum,Koloseum,Koloseumi,Kolosseum,Kolosseum,Kolosseum,Kolosseum"

In [9]:
spark.sql("SELECT GeoDbscan(longitude, latitude, 150, 50) AS cluster FROM posts WHERE \
          GeoData(description,'"+keywords+"') IS NOT NULL AND latitude > 0 AND longitude > 0").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|cluster                                                                                                                                                                                                                                                                                                                                                                                                                                                      